<a href="https://colab.research.google.com/github/melihkurtaran/DNA_hackathon/blob/template_matching/MainCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install gif2numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for gif2numpy: filename=gif2numpy-1.3-py2.py3-none-any.whl size=8392 sha256=1c86c0201ef855ce0a12f2d2083019dfd20e089c676df8fb0686cdf172170418
  Stored in directory: /root/.cache/pip/wheels/7d/70/75/7e42205c427015e223944d47ea89e01dc779fb49ba664fb743
Successfully built gif2numpy


In [2]:
!unzip /content/drive/MyDrive/EDISS_winter_school/aemet.zip -d /content/drive/MyDrive/EDISS_winter_school/data

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/EDISS_winter_school/data/aemet/10min/ba/20220123/aemet_ba_202201230640.gif  
  inflating: /content/drive/MyDrive/EDISS_winter_school/data/__MACOSX/aemet/10min/ba/20220123/._aemet_ba_202201230640.gif  
  inflating: /content/drive/MyDrive/EDISS_winter_school/data/aemet/10min/ba/20220123/aemet_ba_202201230720.gif  
  inflating: /content/drive/MyDrive/EDISS_winter_school/data/__MACOSX/aemet/10min/ba/20220123/._aemet_ba_202201230720.gif  
  inflating: /content/drive/MyDrive/EDISS_winter_school/data/aemet/10min/ba/20220123/aemet_ba_202201231600.gif  
  inflating: /content/drive/MyDrive/EDISS_winter_school/data/__MACOSX/aemet/10min/ba/20220123/._aemet_ba_202201231600.gif  
  inflating: /content/drive/MyDrive/EDISS_winter_school/data/aemet/10min/ba/20220123/aemet_ba_202201230250.gif  
  inflating: /content/drive/MyDrive/EDISS_winter_school/data/__MACOSX/aemet/10min/ba/20220123/._aemet_ba_202201230250.gif  
 

In [6]:
import gif2numpy
import cv2
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image,ImageDraw
import os
%matplotlib inline
import numpy as np

In [11]:
data_path = '/content/drive/MyDrive/EDISS_winter_school/data/aemet/10min/'
data = {}
for state in os.listdir(data_path):
  if i[0] =='.':
    continue
  data[state] = {}
  for i in os.listdir(os.path.join(data_path,state)):
    if i[0] =='.':
      continue
    if i not in data[state]:
      data[state][i] = []
    data[state][i].append(os.path.join(data_path,state,i))



NotADirectoryError: ignored